In [ ]:
from gdpyt import GdpytImageCollection, GdpytCalibrationSet
from gdpyt.utils.evaluation import GdpytPerformanceEvaluation
from os.path import join
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib notebook

Datasets

In [ ]:
datasets = ['DS_Grid_N50_Nc10_5um_Sigma1_f365', # 20 calib images
            'DS_Grid_N50_Nc20_5um_Sigma1_f365', # 50 calib images
            'DS_Grid_N50_Nc50_5um_Sigma1_f365' # 100 calib images
           ]
# folder = join(r'C:\Users\silus\UCSB\master_thesis\python_stuff\gdpyt\tests\test_synthetic', datasets[1])
folder = join(r'C:\Users\silus\UCSB\master_thesis\python_stuff\gdpyt\tests\test_synthetic\Rossi_DS1_Nc50_Sigma0')
calib_img_path = join(folder, 'calibration_images') # Calibration images (10 x 10 particles )
test_img_path = join(folder, 'images') # Test images (10 x 10 particles at a random z height)
test_txt_path = join(folder, 'input') # Ground truth (text files with coordinates of particles)

Load calibration images

In [ ]:
filetype = '.tif'
# Add filters if wanted for error assessment
processing = {
    'cv2.convertScaleAbs': {'args': [], 'kwargs': dict(alpha=0.1, beta=0)},
    #'cv2.GaussianBlur': {'args': [(11, 11), 5]},
    'cv2.medianBlur': {'args': [5]}
}
threshold = {
    'manual': [24]
           #'otsu': [] 
}

calib_col = GdpytImageCollection(calib_img_path, filetype, processing_specs=processing, thresholding_specs=threshold,
                                 min_particle_size=5, max_particle_size=5000, shape_tol=0.5)
calib_col.uniformize_particle_ids(threshold=30)
name_to_z = {}
for image in calib_col.images.values():
    name_to_z.update({image.filename: float(image.filename.split('_')[-1].split(filetype)[0])})

# Dilate: dilate images to have room to slide template and find ideal position
calib_set = calib_col.create_calibration(name_to_z, dilate=True) 

In [ ]:
fig = calib_col.plot(raw=False, draw_particles=True)
fig.subplots_adjust(hspace=0.1)
fig.show()

Load test images

In [ ]:
collection = GdpytImageCollection(test_img_path, filetype, processing_specs=processing, thresholding_specs=threshold,
                                 min_particle_size=5, max_particle_size=5000, shape_tol=0.5)
collection.uniformize_particle_ids(baseline=calib_set, threshold=30)

In [ ]:
fig = collection.plot(raw=False, draw_particles=True)
fig.show()

Infer z

In [ ]:
collection.infer_z(calib_set, function='znccorr')

In [ ]:
###### Coordinates of all particles on the 36th image
collection[36].particles[13].similarity_curve

Test the performance and make error plot

In [ ]:
# Evaluate performance #############################################################################################
perf_eval = GdpytPerformanceEvaluation(collection, test_txt_path)
perf_eval.sigma_z_local()

In [ ]:
calib_set.calibration_stacks[10]

In [ ]:
sz_loc = perf_eval.sigma_z_local()
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(sz_loc['z'], sz_loc['sigma_z_local'])
ax.set_ylabel(r'$\sigma_z$ [$\mu$ m]', fontsize=12)
ax.set_xlabel('Z [$\mu$ m]')
ax2 = ax.twinx()
ax2.plot(sz_loc['z'], sz_loc['sigma_z_local'] / (abs(sz_loc['z'].max()) + abs(sz_loc['z'].min())))
ax2.set_ylabel(r'$\sigma_z / h$', fontsize=16)